In [1]:
import gc
import numpy as np
import pandas as pd

from category_encoders import BinaryEncoder
from sklearn import preprocessing
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree._tree import TREE_LEAF

gc.enable()

pd.options.display.max_columns = 200
pd.options.display.max_rows = 15

In [4]:
df = pd.read_csv('train_features.csv')
print(df.shape)
df.head()

(37745, 103)


,id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method
0,43373,NaN,12000,12000,36 months,16.02%,422.01,C,C5,Driver,4 years,MORTGAGE,81000.0,NaN,NaN,debt_consolidation,Debt consolidation,280xx,NC,12.76,0,Dec-2005,1,48.0,NaN,7,0,4912,23.5%,15,w,0,48.0,Individual,NaN,NaN,0,0,174783,1,2,0,4,13.0,33668,82.0,2,5,2962,63.0,20900,1,1,2,9,24969.0,588.0,89.3,0,0,145.0,23,1,1,1,8.0,NaN,1.0,NaN,2,2,2,2,2,9,4,5,2,7,0.0,0,0,2,92.9,100.0,0,0,206618,38580,5500,40863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1,24414,NaN,6300,6300,36 months,14.07%,215.54,C,C3,GMP Lead,4 years,RENT,39000.0,NaN,NaN,debt_consolidation,Debt consolidation,920xx,CA,21.42,0,Mar-2012,1,NaN,NaN,10,0,11876,59.4%,12,w,0,NaN,Individual,NaN,NaN,0,0,31046,0,1,0,2,14.0,19170,83.0,3,8,5153,72.0,20000,1,0,1,10,3105.0,6877.0,59.3,0,0,17.0,72,8,8,0,8.0,NaN,6.0,NaN,0,6,7,7,8,2,9,10,7,10,0.0,0,0,3,100.0,42.9,0,0,43140,31046,16900,23140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
2,46723,NaN,4500,4500,36 months,7.21%,139.38,A,A3,Accounts,3 years,RENT,78000.0,NaN,NaN,debt_consolidation,Debt consolidation,906xx,CA,2.17,0,May-2000,2,NaN,NaN,13,0,1715,5.2%,19,w,0,NaN,Individual,NaN,NaN,0,0,35329,0,8,1,2,10.0,33614,106.0,1,1,1196,54.0,33300,4,0,3,3,2718.0,21585.0,7.4,0,0,166.0,218,10,10,0,31.0,NaN,1.0,NaN,0,3,3,3,4,13,5,6,3,13,0.0,0,0,2,100.0,0.0,0,0,65092,35329,23300,31792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
3,24878,NaN,12000,12000,36 months,9.44%,384.06,B,B1,Client Processing Associate,10+ years,MORTGAGE,62000.0,NaN,NaN,debt_consolidation,Debt consolidation,441xx,OH,13.76,0,Feb-2003,0,NaN,NaN,22,0,9404,16.3%,33,w,0,NaN,Individual,NaN,NaN,0,0,75160,4,1,0,2,13.0,17123,67.0,6,9,6898,32.0,57600,3,1,2,12,3579.0,33602.0,17.0,0,0,162.0,179,1,1,1,1.0,NaN,5.0,NaN,0,1,3,9,13,7,20,25,3,22,0.0,0,0,7,100.0,0.0,0,0,133065,26527,40500,25465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
4,39038,NaN,12000,12000,36 months,22.35%,460.47,D,D5,Construction Operator,10+ years,MORTGAGE,70000.0,NaN,NaN,debt_consolidation,Debt consolidation,751xx,TX,14.04,4,Nov-2001,2,13.0,NaN,18,0,17895,24.2%,45,w,0,13.0,Individual,NaN,NaN,0,7218,113702,9,2,2,4,6.0,16711,68.0,9,10,5373,35.0,73900,3,6,3,14,6317.0,32863.0,24.1,0,0,159.0,201,0,0,1,0.0,62.0,3.0,62.0,6,9,11,11,14,22,15,20,11,18,0.0,0,2,11,82.2,9.1,0,0,192842,34606,43300,24499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash


In [5]:
cell_count = np.product(df.shape)

missing_count  = df.isnull().sum()

total_missing = missing_count.sum()

print("Missing values in percentile format:", round(((total_missing/cell_count) * 100),
                                                   2), '%')

Missing values in percentile format: 18.16 %


# Remove useless columns
Data is missing entirely for some columns, so we remove them.

In [6]:
drop_columns = missing_count[missing_count == 37745].index.to_list()
drop_columns

['member_id', 'url', 'desc']

In [7]:
df = df.drop(labels=drop_columns, axis=1)

In [8]:
nulls = df.isnull().sum()
nulls

# I've decided to keep the sec_app data even though a large majority of it is missing. 
# data description https://docs.google.com/spreadsheets/d/1R_0o6LWrI93x2ge15jcKZ_auyRYTwH_pV6DLj3ej-ZQ/edit#gid=1444157382
# makes it seem important, indictation of existing debt is usually a good indication towards not being able to repay further

id                                         0
loan_amnt                                  0
funded_amnt                                0
term                                       0
int_rate                                   0
installment                                0
grade                                      0
                                       ...  
sec_app_revol_util                     33106
sec_app_open_act_il                    33007
sec_app_num_rev_accts                  33007
sec_app_chargeoff_within_12_mths       33007
sec_app_collections_12_mths_ex_med     33007
sec_app_mths_since_last_major_derog    36077
disbursement_method                        0
Length: 100, dtype: int64

# Encoding object categories with Binary Encoding
We want to use sklearn's tree models, and we need to make to label these string variables.
We use the wonderful module category_encoders, an exstension of sklearn

In [9]:
to_encode = df.select_dtypes(include=['object']).columns.tolist()
print(to_encode)

['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'application_type', 'sec_app_earliest_cr_line', 'disbursement_method']


In [91]:
binary_encoder = BinaryEncoder(cols=to_encode).fit(df)
converted_df = binary_encoder.transform(df)

In [92]:
print(converted_df.dtypes.value_counts())
converted_df.head()

int64      162
float64     32
dtype: int64


,term_0,term_1,int_rate_0,int_rate_1,int_rate_2,int_rate_3,int_rate_4,int_rate_5,int_rate_6,int_rate_7,grade_0,grade_1,grade_2,grade_3,sub_grade_0,sub_grade_1,sub_grade_2,sub_grade_3,sub_grade_4,sub_grade_5,sub_grade_6,emp_title_0,emp_title_1,emp_title_2,emp_title_3,emp_title_4,emp_title_5,emp_title_6,emp_title_7,emp_title_8,emp_title_9,emp_title_10,emp_title_11,emp_title_12,emp_title_13,emp_title_14,emp_length_0,emp_length_1,emp_length_2,emp_length_3,emp_length_4,home_ownership_0,home_ownership_1,home_ownership_2,purpose_0,purpose_1,purpose_2,purpose_3,purpose_4,title_0,title_1,title_2,title_3,title_4,zip_code_0,zip_code_1,zip_code_2,zip_code_3,zip_code_4,zip_code_5,zip_code_6,zip_code_7,zip_code_8,zip_code_9,zip_code_10,addr_state_0,addr_state_1,addr_state_2,addr_state_3,addr_state_4,addr_state_5,addr_state_6,earliest_cr_line_0,earliest_cr_line_1,earliest_cr_line_2,earliest_cr_line_3,earliest_cr_line_4,earliest_cr_line_5,earliest_cr_line_6,earliest_cr_line_7,earliest_cr_line_8,earliest_cr_line_9,earliest_cr_line_10,revol_util_0,revol_util_1,revol_util_2,revol_util_3,revol_util_4,revol_util_5,revol_util_6,revol_util_7,revol_util_8,revol_util_9,revol_util_10,revol_util_11,initial_list_status_0,initial_list_status_1,application_type_0,application_type_1,sec_app_earliest_cr_line_0,sec_app_earliest_cr_line_1,sec_app_earliest_cr_line_2,sec_app_earliest_cr_line_3,sec_app_earliest_cr_line_4,sec_app_earliest_cr_line_5,sec_app_earliest_cr_line_6,sec_app_earliest_cr_line_7,sec_app_earliest_cr_line_8,sec_app_earliest_cr_line_9,disbursement_method_0,disbursement_method_1,id,loan_amnt,funded_amnt,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,43373,12000,12000,422.01,81000.0,12.76,0,1,48.0,99999.0,7,0,4912,15,0,48.0,99999.0,99999.0,0,0,174783,1,2,0,4,13.0,33668,82.0,2,5,2962,63.0,20900,1,1,2,9,24969.0,588.0,89.3,0,0,145.0,23,1,1,1,8.0,99999.0,1.0,99999.0,2,2,2,2,2,9,4,5,2,7,0.0,0,0,2,92.9,100.0,0,0,206618,38580,5500,40863,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0
1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,24414,6300,6300,215.54,39000.0,21.42,0,1,99999.0,99999.0,10,0,11876,12,0,99999.0,99999.0,99999.0,0,0,31046,0,1,0,2,14.0,19170,83.0,3,8,5153,72.0,20000,1,0,1,10,3105.0,6877.0,59.3,0,0,17.0,72,8,8,0,8.0,99999.0,6.0,99999.0,0,6,7,7,8,2,9,10,7,10,0.0,0,0,3,100.0,42.9,0,0,43

# Bring in label data and look at most import features from tree

In [93]:
df_labels = pd.read_csv('train_labels.csv')
converted_df.fillna(99999, inplace=True)

In [ ]:
tree_classifier = DecisionTreeClassifier(max_depth = 100)
tree_classifier.fit(converted_df, df_labels)

In [95]:
feature_ranks = dict(zip(converted_df.columns, tree_classifier.feature_importances_))

In [96]:
feature_ranks

{'term_0': 0.0,
 'term_1': 0.0,
 'int_rate_0': 0.0,
 'int_rate_1': 0.0,
 'int_rate_2': 0.0,
 'int_rate_3': 0.0,
 'int_rate_4': 0.0,
 'int_rate_5': 0.0,
 'int_rate_6': 0.0,
 'int_rate_7': 0.0,
 'grade_0': 0.0,
 'grade_1': 0.6187665844580301,
 'grade_2': 0.23933700195265314,
 'grade_3': 0.0,
 'sub_grade_0': 0.0,
 'sub_grade_1': 0.0,
 'sub_grade_2': 0.0,
 'sub_grade_3': 0.0,
 'sub_grade_4': 0.0,
 'sub_grade_5': 0.0,
 'sub_grade_6': 0.0,
 'emp_title_0': 0.0,
 'emp_title_1': 0.0,
 'emp_title_2': 0.0,
 'emp_title_3': 0.0,
 'emp_title_4': 0.0,
 'emp_title_5': 0.0,
 'emp_title_6': 0.0,
 'emp_title_7': 0.0,
 'emp_title_8': 0.0,
 'emp_title_9': 0.0,
 'emp_title_10': 0.0,
 'emp_title_11': 0.0,
 'emp_title_12': 0.0,
 'emp_title_13': 0.0,
 'emp_title_14': 0.0,
 'emp_length_0': 0.0,
 'emp_length_1': 0.0,
 'emp_length_2': 0.0,
 'emp_length_3': 0.0,
 'emp_length_4': 0.0,
 'home_ownership_0': 0.0,
 'home_ownership_1': 0.14189641358931662,
 'home_ownership_2': 0.0,
 'purpose_0': 0.0,
 'purpose_1': 0.0,


In [97]:
feature_ranks = sorted(feature_ranks, key=feature_ranks.get, reverse=True)

In [98]:
top_twenty_features = feature_ranks[:20]
top_twenty_features

['grade_1',
 'grade_2',
 'home_ownership_1',
 'term_0',
 'term_1',
 'int_rate_0',
 'int_rate_1',
 'int_rate_2',
 'int_rate_3',
 'int_rate_4',
 'int_rate_5',
 'int_rate_6',
 'int_rate_7',
 'grade_0',
 'grade_3',
 'sub_grade_0',
 'sub_grade_1',
 'sub_grade_2',
 'sub_grade_3',
 'sub_grade_4']

In [70]:
y_pred_one = tree_classifier.predict_proba(converted_df)

In [75]:
y_pred_one[0][0][0]

0.0

# Breakpoint for prior selected features

In [99]:
new_df = converted_df.filter(top_twenty_features)
new_df.head()

,grade_1,grade_2,home_ownership_1,term_0,term_1,int_rate_0,int_rate_1,int_rate_2,int_rate_3,int_rate_4,int_rate_5,int_rate_6,int_rate_7,grade_0,grade_3,sub_grade_0,sub_grade_1,sub_grade_2,sub_grade_3,sub_grade_4
0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
2,0,1,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
3,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
4,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1


In [100]:
tree_classifier = DecisionTreeClassifier()
tree_classifier.fit(new_df, df_labels['charged_off'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [101]:
feature_ranks = dict(zip(new_df.columns, tree_classifier.feature_importances_))
feature_ranks

{'grade_1': 0.29489580076217303,
 'grade_2': 0.11693390145934622,
 'home_ownership_1': 0.10891852665967401,
 'term_0': 0.018362498617464963,
 'term_1': 0.029502880286955104,
 'int_rate_0': 0.0,
 'int_rate_1': 0.057839917711322075,
 'int_rate_2': 0.05623266301945564,
 'int_rate_3': 0.01968144887319467,
 'int_rate_4': 0.02992645403481797,
 'int_rate_5': 0.08689697791559321,
 'int_rate_6': 0.04847186541754462,
 'int_rate_7': 0.028770529623261553,
 'grade_0': 0.0,
 'grade_3': 0.04763533160030156,
 'sub_grade_0': 0.0,
 'sub_grade_1': 0.0002930570625290786,
 'sub_grade_2': 0.017901620010724425,
 'sub_grade_3': 0.009121685208839663,
 'sub_grade_4': 0.028614841736802104}

In [102]:
y_true = df_labels['charged_off']

In [103]:
accuracy_score(y_true, tree_classifier.predict(new_df), normalize=False)

32204

# Create test df and process as before

In [113]:
test_df = pd.read_csv('test_features.csv')
to_encode = test_df.select_dtypes(include=['object']).columns.tolist()
binary_encoder = BinaryEncoder(cols=to_encode).fit(test_df)
test_df = binary_encoder.transform(test_df)
test_df.fillna(99999, inplace=True)
test_df_final = test_df.filter(top_twenty_features)

In [115]:
probs = tree_classifier.predict_proba(test_df_final)[:,1]
probs = probs.tolist()
probs

[0.20300751879699247,
 0.01744186046511628,
 0.16030534351145037,
 0.0945945945945946,
 0.09473684210526316,
 0.0,
 0.020833333333333332,
 0.0,
 0.4918032786885246,
 0.0,
 0.0,
 0.0,
 0.09009009009009009,
 0.2037037037037037,
 0.125,
 0.33783783783783783,
 0.0,
 0.007194244604316547,
 0.4918032786885246,
 0.020833333333333332,
 0.0,
 0.038314176245210725,
 0.33783783783783783,
 0.14814814814814814,
 0.03333333333333333,
 0.03571428571428571,
 0.16666666666666666,
 0.17708333333333334,
 0.25,
 0.3291139240506329,
 0.1724137931034483,
 0.0,
 0.13559322033898305,
 0.2037037037037037,
 0.0,
 0.0945945945945946,
 0.33783783783783783,
 0.14814814814814814,
 0.4186046511627907,
 0.20245398773006135,
 1.0,
 0.05555555555555555,
 0.3333333333333333,
 0.0,
 0.14285714285714285,
 0.3546099290780142,
 0.2746478873239437,
 0.33783783783783783,
 0.0,
 0.0,
 0.5,
 0.019138755980861243,
 0.0,
 0.0,
 0.0,
 0.0,
 0.4444444444444444,
 0.0,
 0.06511627906976744,
 0.03076923076923077,
 0.0,
 0.0,
 0.055555

In [116]:
X = pd.DataFrame(test_df.id)

In [117]:
X['charged_off'] = probs

In [118]:
X = X.set_index('id')
X.to_csv('submission.csv')

# Reduce the complexity of the model with pruning

In [ ]:
class Pruner:
    def check_leaf(inner_tree, index):
        """Check whether node is leaf node"""
        return (inner_tree.children_left[index] == TREE_LEAF and 
                inner_tree.children_right[index] == TREE_LEAF)

    def index_pruner(inner_tree, decisions, index=0):
        """ 
        Function to parse trees bottom up, helper function for prune duplicate leaves.
        Do not use directly
        """
        if not check_leaf(inner_tree, inner_tree.children_left[index]):
            index_pruner(inner_tree, decisions, inner_tree.children_left[index])
        if not check_leaf(inner_tree, inner_tree.children_right[index]):
            index_pruner(inner_tree, decisions, inner_tree.children_right[index])

        # Prune children if both children are leaves now and make the same decision:     
        if (check_leaf(inner_tree, inner_tree.children_left[index]) and
            check_leaf(inner_tree, inner_tree.children_right[index]) and
            (decisions[index] == decisions[inner_tree.children_left[index]]) and 
            (decisions[index] == decisions[inner_tree.children_right[index]])):
            # turn node into a leaf by "unlinking" its children
            inner_tree.children_left[index] = TREE_LEAF
            inner_tree.children_right[index] = TREE_LEAF
            ##print("Pruned {}".format(index))

    def prune_duplicate_leaves(mdl):
        # Remove leaves if both 
        decisions = mdl.tree_.value.argmax(axis=2).flatten().tolist() # Decision for each node
        index_pruner(mdl.tree_, decisions)